# Introduction

A basic random forest, built on top of previously used [decision trees](decision_tree.ipynb).

# Setup

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [2]:
# Load data

df = pd.read_csv("../../data/Iris.csv")
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
# Drop Id

df = df.drop(columns=["Id"])

In [4]:
# Seperate features and labels

features = df.drop(columns=["Species"]).values
labels = df["Species"].unique()
labels_encoded = np.argmax(pd.get_dummies(df["Species"]).astype(int).values, axis=1)

features[:5], labels_encoded[:5], labels

(array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2]]),
 array([0, 0, 0, 0, 0], dtype=int64),
 array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object))

In [5]:
# Train and test splits

X_train, X_test, y_train, y_test = train_test_split(features, labels_encoded, test_size=0.2)

In [6]:
print(f'Training dataset size: {len(X_train)}')
print(f'Test dataset size: {len(X_test)}')

Training dataset size: 120
Test dataset size: 30


# Model

In [13]:
def bootstrap_sample(X, y):
    n_samples = X.shape[0]
    indices = np.random.choice(n_samples, size=n_samples, replace=True)
    return X[indices], y[indices]

In [14]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

In [15]:
def gini(y):
    classes, counts = np.unique(y, return_counts=True)
    probs = counts / counts.sum()
    return 1 - np.sum(probs ** 2)

In [16]:
def split_data(X, y, feature_index, threshold):
    left_mask = X[:, feature_index] <= threshold
    right_mask = X[:, feature_index] > threshold
    return X[left_mask], y[left_mask], X[right_mask], y[right_mask]

In [17]:
def best_split(X, y, max_features=None):
    n_samples, n_features = X.shape
    if max_features is None:
        feature_indices = range(n_features)
    else:
        feature_indices = np.random.choice(n_features, max_features, replace=False)
    
    best_gini = float("inf")
    best_index = None
    best_threshold = None

    for feature_index in feature_indices:
        thresholds = np.unique(X[:, feature_index])
        for threshold in thresholds:
            X_left, y_left, X_right, y_right = split_data(X, y, feature_index, threshold)
            
            if len(y_left) == 0 or len(y_right) == 0:
                continue
            
            gini_left = gini(y_left)
            gini_right = gini(y_right)
            weighted_gini = (len(y_left) * gini_left + len(y_right) * gini_right) / len(y)

            if weighted_gini < best_gini:
                best_gini = weighted_gini
                best_index = feature_index
                best_threshold = threshold

    return best_index, best_threshold


In [18]:
def build_tree(X, y, max_depth=5, min_samples_split=2, max_features=None, depth=0):
    n_samples, n_features = X.shape
    n_labels = len(np.unique(y))

    # Stop criteria
    if depth >= max_depth or n_labels <= 1 or n_samples < min_samples_split:
        leaf_value = np.bincount(y).argmax() # Most common label
        return Node(value=leaf_value)
    
    feature_index, threshold = best_split(X, y, max_features)
    if feature_index == None:
        leaf_value = np.bincount(y).argmax() # Most common label
        return Node(value=leaf_value)
    
    X_left, y_left, X_right, y_right = split_data(X, y, feature_index, threshold)
    left_subtree = build_tree(X_left, y_left, max_depth=max_depth, min_samples_split=min_samples_split, max_features=max_features, depth=depth+1)
    right_subtree = build_tree(X_right, y_right, max_depth=max_depth, min_samples_split=min_samples_split, max_features=max_features, depth=depth+1)

    return Node(feature_index=feature_index, threshold=threshold, left=left_subtree, right=right_subtree)


In [19]:
def build_forest(X, y, n_trees=10, max_depth=5, min_samples_split=2, max_features=None):
    trees = []

    for i in range(n_trees):
        X_sample, y_sample = bootstrap_sample(X, y)
        trees.append(build_tree(X_sample, y_sample,
                                max_depth=max_depth,
                                min_samples_split=min_samples_split,
                                max_features=max_features))
        
    return trees
        

In [20]:
def predict_tree(sample, tree: Node):
    # This is a leaf
    if tree.value != None:
        return tree.value
    
    if sample[tree.feature_index] <= tree.threshold:
        return predict_tree(sample, tree.left)
    else:
        return predict_tree(sample, tree.right) 

In [21]:
def predict_dataset_tree(X, tree):
    return np.array([predict_tree(sample, tree) for sample in X])

In [22]:
def predict_dataset_forest(X, forest):
    tree_preds = np.array([predict_dataset_tree(X, tree) for tree in forest])
    return np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=tree_preds) # majority vote

In [23]:
def calculate_accuracy(y, y_preds):
    return (y_preds == y).astype(int).sum() / len(y)

In [24]:
forest = build_forest(X_test, y_test, n_trees=10, max_depth=5, min_samples_split=2, max_features=None)

In [40]:
class RandomForestMetric:
    def __init__(self, n_trees, max_depth, train_accuracy, test_accuracy):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.train_accuracy = train_accuracy
        self.test_accuracy = test_accuracy

In [41]:
metrics = []
tree_range = range(5, 16)
depth_range = range(5, 16)

for n_trees in tree_range:
    for depth in depth_range:
        forest = build_forest(X_train, y_train, n_trees=n_trees, max_depth=depth, min_samples_split=2, max_features=None)

        train_preds = predict_dataset_forest(X_train, forest)
        train_acc = calculate_accuracy(y_train, train_preds)

        test_preds = predict_dataset_forest(X_test, forest)
        test_acc = calculate_accuracy(y_test, test_preds)

        metrics.append(RandomForestMetric(n_trees, depth, train_acc, test_acc))

        print(f"Trees: {n_trees} | Max Depth: {depth} | Train Acc: {train_acc*100:.2f}% | Test Acc: {test_acc*100:.2f}%")
    

Trees: 5 | Max Depth: 5 | Train Acc: 98.33% | Test Acc: 93.33%
Trees: 5 | Max Depth: 6 | Train Acc: 100.00% | Test Acc: 90.00%
Trees: 5 | Max Depth: 7 | Train Acc: 100.00% | Test Acc: 90.00%
Trees: 5 | Max Depth: 8 | Train Acc: 100.00% | Test Acc: 93.33%
Trees: 5 | Max Depth: 9 | Train Acc: 99.17% | Test Acc: 93.33%
Trees: 5 | Max Depth: 10 | Train Acc: 98.33% | Test Acc: 90.00%
Trees: 5 | Max Depth: 11 | Train Acc: 100.00% | Test Acc: 93.33%
Trees: 5 | Max Depth: 12 | Train Acc: 99.17% | Test Acc: 93.33%
Trees: 5 | Max Depth: 13 | Train Acc: 99.17% | Test Acc: 93.33%
Trees: 5 | Max Depth: 14 | Train Acc: 97.50% | Test Acc: 90.00%
Trees: 5 | Max Depth: 15 | Train Acc: 99.17% | Test Acc: 86.67%
Trees: 6 | Max Depth: 5 | Train Acc: 99.17% | Test Acc: 90.00%
Trees: 6 | Max Depth: 6 | Train Acc: 99.17% | Test Acc: 93.33%
Trees: 6 | Max Depth: 7 | Train Acc: 98.33% | Test Acc: 96.67%
Trees: 6 | Max Depth: 8 | Train Acc: 98.33% | Test Acc: 93.33%
Trees: 6 | Max Depth: 9 | Train Acc: 100.00% 

In [44]:
best_idx = None
best_acc = float("-inf")

for i, metric in enumerate(metrics):
    if metric.test_accuracy > best_acc:
        best_idx = i
        best_acc = metric.test_accuracy

metric = metrics[best_idx]
print("Forest with the highest accuracy on the test dataset:")
print(f"Trees: {metric.n_trees} | Max Depth: {metric.max_depth} | Train Acc: {metric.train_accuracy*100:.2f}% | Test Acc: {metric.test_accuracy*100:.2f}%")

Forest with the highest accuracy on the test dataset:
Trees: 6 | Max Depth: 7 | Train Acc: 98.33% | Test Acc: 96.67%


# Conclusion

The random forest algorithm perform better than the decision tree, as expected. It can achieve up to 96.67% accuracy on the test dataset. Increasing number of trees and maximum depth doesn't seem to improve performance by a large margin.

Metrics:
- Highest accuracy achieved on test dataset: 96.67%